In [104]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

In [105]:
client_id = '~~~~'
client_pw = '~~~~'

In [106]:
api_url = 'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords='
print_type = '&output=json'

In [158]:
finance = pd.read_csv('jeju_financial_life_data.csv')

In [129]:
data = pd.read_csv('before_finance.csv')

In [73]:
finance.columns

Index(['zip_cd', 'year_month', 'x_axis', 'y_axis', 'sex', 'age', 'job_majorc',
       'job_smallc', 'job_public', 'job_profession', 'job_self', 'job_none',
       'job_other', 'avg_income', 'med_income', 'avg_spend',
       'avg_foreign_spend', 'avg_debt', 'avg_debt_credit', 'avg_debt_noneb',
       'avg_debt_mortgage', 'avg_debt_deposit', 'avg_debt_collateral',
       'avg_credit_rat', 'medium_resid_rat', 'large_resid_rat',
       'vehicle_own_rat'],
      dtype='object')

In [132]:
data.columns

Index(['Unnamed: 0', '1012a', '1012b', '10~11_ride', '10~11_takeoff',
       '11~12_ride', '11~12_takeoff', '1820_r_mean', '1820_r_sum',
       '1820_rs_mean',
       ...
       'weekdaymean', 'in_out_mean', 'adult', 'kids', 'teen', 'elder',
       'adult_prop', 'kids_prop', 'teen_prop', 'elder_prop'],
      dtype='object', length=143)

In [159]:
finance = finance[['x_axis', 'y_axis']]
finance.head()

,x_axis,y_axis
0,126.565889,33.513210
1,126.497121,33.247225
2,126.497121,33.247225
3,126.497121,33.247225
4,126.497121,33.247225


In [110]:
data = data[['latitude', 'longitude']]
data.head()

,latitude,longitude
0,33.48990,126.49373
1,33.48944,126.48508
2,33.48181,126.47352
3,33.50577,126.49252
4,33.25579,126.41260


In [160]:
finance = finance.rename(columns={'x_axis':'longitude', 'y_axis':'latitude'})
finance.head()

,longitude,latitude
0,126.565889,33.513210
1,126.497121,33.247225
2,126.497121,33.247225
3,126.497121,33.247225
4,126.497121,33.247225


In [161]:
finance = finance.drop_duplicates()
data = data.drop_duplicates()

In [162]:
def change(x):
    return str(x)[0:9]

In [163]:
data['longitude'] = data['longitude'].apply(change)
data['latitude'] = data['latitude'].apply(change)
finance['longitude'] = finance['longitude'].apply(change)
finance['latitude'] = finance['latitude'].apply(change)

In [164]:
data['query']= data['longitude'] + ',' + data['latitude']
finance['query']=finance['longitude'] + ',' + finance['latitude']

In [165]:
data.head(20)

,Unnamed: 0,1012a,1012b,10~11_ride,10~11_takeoff,11~12_ride,11~12_takeoff,1820_r_mean,1820_r_sum,1820_rs_mean,...,in_out_mean,adult,kids,teen,elder,adult_prop,kids_prop,teen_prop,elder_prop,query
0,0,8.0,0.0,2.0,0.0,6.0,0.0,3.104381,2409.0,1.466667,...,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,"126.49373,33.4899"
1,1,11.0,0.0,5.0,0.0,6.0,0.0,3.104381,2409.0,5.366667,...,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,"126.48508,33.48944"
2,2,0.0,0.0,0.0,0.0,0.0,0.0,3.104381,2409.0,1.600000,...,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,"126.47352,33.481809"
3,3,30.0,0.0,14.0,0.0,16.0,0.0,3.104381,2409.0,53.766667,...,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,"126.49252,33.50577"
4,4,0.0,0.0,0.0,0.0,0.0,0.0,3.104381,2409.0,0.153846,...,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,"126.4126,33.255790"
5,5,1.0,1.0,1.0,1.0,0.0,0.0,3.104381,2409.0,0.407407,...,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,"126.40442,33.25862"
6,6,2.0,6.0,2.0,1.0,0.0,5.0,3.104381,2409.0,4.724138,...,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,"126.42472,33.24309"
7,7,0.0,0.0,0.0,0.0,0.0,0.0,3.104381,2409.0,0.185185,...,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,"126.37082,33.26598"
8,8,1.0,1.0,0.0,1.0,1.0,0.0,3.104381,2409.0,1.133333,...,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,"126.47827,33.23603"
9,9,0.0,2.0,0.0,0.0,0.0,2.0,3.104381,2409.0,0.000000,...,1.0,8158.0,72.0,38.0,0.0,0.986696,0.008708,0.004596,0.0,"126.5645,33.23977"


In [166]:
data_query = list(data['query'])
data_query2 = list(finance['query'])

In [167]:
data_query2

['126.56588,33.513209',
 '126.49712,33.247224',
 '126.43490,33.260916',
 '126.41823,33.467606',
 '126.79059,33.532487',
 '126.55473,33.501844',
 '126.54674,33.508162',
 '126.53386,33.453579',
 '126.54025,33.505111',
 '126.59531,33.486649',
 '126.53770,33.495873',
 '126.28705,33.279439',
 '126.62040,33.535406',
 '126.52124,33.505112',
 '126.54611,33.514339',
 '126.52121,33.507347',
 '126.49909,33.236161',
 '126.54130,33.498186',
 '126.55265,33.515265',
 '126.49532,33.478716',
 '126.44113,33.471023',
 '126.48412,33.483036',
 '126.87753,33.381261',
 '126.49150,33.480087',
 '126.39631,33.251423',
 '126.52380,33.494685',
 '126.95444,33.508542',
 '126.41113,33.247537',
 '126.55760,33.512904',
 '126.25420,33.224751',
 '126.49328,33.518696',
 '126.66666,33.540716',
 '126.79969,33.317535',
 '126.51409,33.503395',
 '126.51930,33.496822',
 '126.57419,33.250336',
 '126.48340,33.480725',
 '126.56360,33.249760',
 '126.56436,33.508281',
 '126.58190,33.520292',
 '126.52730,33.497353',
 '126.27879,33.4

In [168]:
add_list = data_query
add_list2 = data_query2

In [169]:
len(add_list2)

474

In [170]:
geo_coordi2 = []
for add in add_list2:
    add_urlenc = parse.quote(add) # 주소를 아스키코드에 맞게 인코딩해주는 작업
    url = api_url + add_urlenc + print_type
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        sigugun = None
        dongmyun = None
    else:
        rescode = response.getcode() # 정상이면 200 리턴
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)
            if 'results' in response_body:
                try:
                    sigugun = response_body['results'][0]['region']['area2']['name']
                    dongmyun = response_body['results'][0]['region']['area3']['name']
                    print('Success!')
                except:
                    sigugun = None
                    dongmyun = None
            else:
                print("'result' not exist!")
                sigugun = None
                dongmyun = None
        else:
            print('Response error code : %d' % rescode)
            sigugun = None
            dongmyun = None

    geo_coordi2.append([add, sigugun, dongmyun])

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
S

In [171]:
geo_coordi2

[['126.56588,33.513209', '제주시', '화북일동'],
 ['126.49712,33.247224', '서귀포시', '강정동'],
 ['126.43490,33.260916', '서귀포시', '중문동'],
 ['126.41823,33.467606', '제주시', '애월읍'],
 ['126.79059,33.532487', '제주시', '구좌읍'],
 ['126.55473,33.501844', '제주시', '일도이동'],
 ['126.54674,33.508162', '제주시', '건입동'],
 ['126.53386,33.453579', '제주시', '오등동'],
 ['126.54025,33.505111', '제주시', '일도이동'],
 ['126.59531,33.486649', '제주시', '봉개동'],
 ['126.53770,33.495873', '제주시', '이도이동'],
 ['126.28705,33.279439', '서귀포시', '대정읍'],
 ['126.62040,33.535406', '제주시', '조천읍'],
 ['126.52124,33.505112', '제주시', '삼도이동'],
 ['126.54611,33.514339', '제주시', '건입동'],
 ['126.52121,33.507347', '제주시', '삼도이동'],
 ['126.49909,33.236161', '서귀포시', '강정동'],
 ['126.54130,33.498186', '제주시', '이도이동'],
 ['126.55265,33.515265', '제주시', '화북일동'],
 ['126.49532,33.478716', '제주시', '연동'],
 ['126.44113,33.471023', '제주시', '애월읍'],
 ['126.48412,33.483036', '제주시', '노형동'],
 ['126.87753,33.381261', '서귀포시', '성산읍'],
 ['126.49150,33.480087', '제주시', '연동'],
 ['126.39631,33.251423', '서귀포

In [173]:
len(geo_coordi2)

474

In [130]:
data_coordi = pd.DataFrame(geo_coordi, columns=['coordi', 'si', 'dong'])

In [174]:
data_coordi2 = pd.DataFrame(geo_coordi2, columns=['coordi', 'si', 'dong'])

In [175]:
data_coordi2.head()

,coordi,si,dong
0,"126.56588,33.513209",제주시,화북일동
1,"126.49712,33.247224",서귀포시,강정동
2,"126.43490,33.260916",서귀포시,중문동
3,"126.41823,33.467606",제주시,애월읍
4,"126.79059,33.532487",제주시,구좌읍


In [176]:
life_address = data_coordi2.rename({'coordi':'location'})

In [149]:
data_address.to_csv('data_address.csv', encoding='cp949', index=False)

In [177]:
life_address.to_csv('life_address.csv', encoding='cp949', index=False)